In [42]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import numpy as np


In [43]:
pd.set_option('display.max_colwidth', None)


In [44]:
game_name_list = []
game_genre_list = []
mean_estimation_list = []
#game_platform_list = []

headers = {
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 YaBrowser/23.3.4.603 Yowser/2.5 Safari/537.36'
}

for n in range(1,613):
    url = f'https://www.playground.ru/games?release=all&sort=time&p={n}'
    igor = requests.get(url, headers=headers).text
    soup = BS(igor, 'lxml')

    items = soup.find_all('div', class_='aside')
    for elem in items:
        # ищем имя
        try:
            name = elem.find('div', class_="title").find('a').text
            name = name.replace('\n','')
            game_name_list.append(name)
        except AttributeError:
            game_name_list.append(np.NaN)

        # ищем оценку
        try:
            estimation = elem.find('div', class_='additional').find('span', class_='value').text
            mean_estimation_list.append(float(estimation))
        except AttributeError:
            mean_estimation_list.append(np.NaN)

        # ищем жанры
        try:
            geners = elem.find('div', class_='info').find('div', class_='tags').find_all('a')
            genre_list = []
            for gener in geners:
                genre_list.append(gener.get_text())
                genre_str = ", ".join(genre_list)
            game_genre_list.append(genre_str)
        except AttributeError:
            game_genre_list.append(np.NaN)

        # ищем платформы
        #items = soup.find_all('div', class_='poster')
        #for elem in items:
        #    url2 = 'https://www.playground.ru' + elem.find('a', href=True)['href']
        #    igor2 = requests.get(url2, headers=headers).text
        #    soup2 = BS(igor2, 'lxml')
        #    try:
        #        platforms = soup2.find('span', class_='platform').text
        #        game_platform_list.append(platforms)
        #    except AttributeError:
        #        game_platform_list.append(np.NaN)

game_dt = pd.DataFrame({
    "Title": game_name_list,
    "Genres": game_genre_list,
    #"Platforms": game_platform_list,
    "Estimation": mean_estimation_list
})
display(game_dt.head())
game_dt.info()


,Title,Genres,Estimation
0,Moscow Rush,"Симулятор, Инди, От третьего лица",6.2
1,Beyond Good & Evil 2,"Экшен, Адвенчура, Открытый мир, Научная фантастика",8.6
2,Asylum,"Адвенчура, Ужасы, От первого лица",8.2
3,SpaceVenture,Адвенчура,7.7
4,The Trench 1916,Экшен,7.1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20802 entries, 0 to 20801
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Title       19578 non-null  object 
 1   Genres      19578 non-null  object 
 2   Estimation  16719 non-null  float64
dtypes: float64(1), object(2)
memory usage: 487.7+ KB


In [45]:
game_dt.to_excel('data/Playground_game.xlsx', index=False)
